## Questão 6

Pesquise sobre redes neurais recorrentes LSTM. Apresente neste estudo aplicações das
LSTM deep learning. Seguem abaixo sugestões de aplicações.

- i) Predição de series temporais (exemplo: predição de palavras no texto, ou predição
de ações na bolsa de valores, etc.)
- ii) Reconhecimento de voz
- iii) Processamento de Linguagem Natural
- iv) Outra aplicações de livre escolha

This study was done by Tiago Dias and its original model can be accessed here [[Link]](https://dadosaocubo.com/processamento-de-linguagem-natural-com-tensorflow/)


In [ ]:
!pip install wordninja

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 541 kB 19.0 MB/s 
  Created wheel for wordninja: filename=wordninja-2.0.0-py3-none-any.whl size=541551 sha256=a1b552c2709978fd312f9884ee539798d4a0254db44772f3be61f90dfd9887b6
  Stored in directory: /root/.cache/pip/wheels/dd/3f/eb/a2692e3d2b9deb1487b09ba4967dd6920bd5032bfd9ff7acfc
Successfully built wordninja


In [ ]:
# Importação das bibliotecas
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
import wordninja
import textblob
import collections
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Importação dos dataset
df = pd.read_csv('https://raw.githubusercontent.com/dadosaocubo/nlp/master/mental_health.csv')

In [ ]:
# Palavras para retirar da análise
stop_words = stopwords.words('english')
# Variável tamanho da base de teste
test_size = 0.1
# Variáveis do modelo
epochs = 10
batch_size = 128

In [ ]:
# Separando palavras juntas
df['text_split'] = df['text'].apply(wordninja.split)
df['text_new'] = df['text_split'].apply(TreebankWordDetokenizer().detokenize)
# Corrigindo palavras incorretas
df['text_new'] = df['text_new'].apply(textblob.TextBlob).apply(textblob.TextBlob.correct).apply(str)

In [ ]:
# Excluindo da descrição os números.
df['text_new'] = df['text_new'].str.replace('[0-9]+', '', regex=True).copy()
# Excluindo da descrição pontuação.
df['text_new'] = df['text_new'].str.replace('[,.:;!?]+', ' ', regex=True).copy()
# Excluindo da descrição caracteres especiais.
df['text_new'] = df['text_new'].str.replace('[/<>()|\+\-\$%&#@\'\"]+', ' ', regex=True).copy()
# Colocando todos os caracteres em caixa baixa.
df['text_new'] = df['text_new'].str.lower().copy()

In [ ]:
# Função para retirar stop words
def tokenize_df(tokenized_words):
  tokenized_words = word_tokenize(tokenized_words)
  stop = [word for word in tokenized_words if word not in stop_words]
  text = TreebankWordDetokenizer().detokenize(stop)
  return text
# Eliminando as stop words
df['text_new'] = df['text_new'].apply(tokenize_df).copy()

In [ ]:
# Selecionando as únicas palavras da variável text_new
df['text_new_split'] = df['text_new'].apply(word_tokenize).copy()
text = list(df.text_new_split)
list_words = [item for sublist in text for item in sublist]
list_words = sorted(list_words)
only_words = set(list_words)

In [ ]:
# Gerando encoder com o vocabulário das palavras
encoder = tfds.deprecated.text.SubwordTextEncoder(vocab_list=only_words)

In [ ]:
# Encode do label
label_encode = LabelEncoder()
target = label_encode.fit_transform(df['label'])

In [ ]:
# Definindo feature e target
x = df['text_new']
y = target
# Dividindo dataset em treino e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size)

In [ ]:
# Função para criação da matriz
def pad_to_size(vec, size):
  zeros = [0] * (size - len(vec))
  vec.extend(zeros)
  return vec
# Função para encode do input
def encode_input(text_new):
  list_x = []
  for text in text_new:
    text_encode = encoder.encode(text)
    text_encode = pad_to_size(text_encode, 64)
    list_x.append(text_encode)
  # Convertendo x em tensor
  input_encode = tf.cast(list_x, tf.float32)
  return input_encode
# Encode do x_train e x_test
x_train = encode_input(x_train)
x_test = encode_input(x_test)

In [ ]:
# Criação do modelo de RNN
inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(encoder.vocab_size, 128)(inputs)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(4, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 128)         123904    
                                                                 
 bidirectional (Bidirectiona  (None, None, 128)        98816     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 4)                 516   

In [ ]:
# Compilando modelo e configurando o processo de treinamento
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="rmsprop",
              metrics=['accuracy'])
# Treinando o modelo
history = model.fit(x_train, y_train, epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(x_test, y_test))

Epoch 1/10
5/5 [==============================] - 14s 962ms/step - loss: 1.3084 - accuracy: 0.4910 - val_loss: 1.0847 - val_accuracy: 0.5968
Epoch 2/10
5/5 [==============================] - 2s 470ms/step - loss: 1.1335 - accuracy: 0.5668 - val_loss: 1.0661 - val_accuracy: 0.5968
Epoch 3/10
5/5 [==============================] - 2s 455ms/step - loss: 1.1150 - accuracy: 0.5686 - val_loss: 1.0410 - val_accuracy: 0.5968
Epoch 4/10
5/5 [==============================] - 2s 472ms/step - loss: 1.0773 - accuracy: 0.5686 - val_loss: 0.9564 - val_accuracy: 0.5968
Epoch 5/10
5/5 [==============================] - 2s 467ms/step - loss: 0.9524 - accuracy: 0.6227 - val_loss: 0.7686 - val_accuracy: 0.7742
Epoch 6/10
5/5 [==============================] - 2s 442ms/step - loss: 0.8708 - accuracy: 0.7184 - val_loss: 0.6711 - val_accuracy: 0.8226
Epoch 7/10
5/5 [==============================] - 2s 470ms/step - loss: 0.6509 - accuracy: 0.7708 - val_loss: 0.6025 - val_accuracy: 0.8065
Epoch 8/10
5/5 [===

In [ ]:
# Testando a qualidade do modelo
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

2/2 [==============================] - 0s 40ms/step - loss: 0.6373 - accuracy: 0.7419
Test Loss: 0.6373046040534973
Test Accuracy: 0.7419354915618896


In [ ]:
# Função para predição
def sample_predict(sample_pred_text):
  encoded_sample_pred_text = encoder.encode(sample_pred_text)
  encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32)
  predictions = model.predict(tf.expand_dims(encoded_sample_pred_text, 0))
  return (predictions)

In [ ]:
# Predição do exemplo
example = 'feel better die happy'
predictions = sample_predict(example)
probabilities = [np.argmax(predictions[0])]
# Retornando os labels
new_label = label_encode.inverse_transform(probabilities)
print('O exemplo "{}" foi classificado como "{}"'.format(example, new_label[0])) 

1/1 [==============================] - 2s 2s/step
O exemplo "feel better die happy" foi classificado como "Depression"
